<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Copy_of_Untitled273shearer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BLOCK 1 — Install + imports

!pip install -q pyscf gemmi cupy-cuda12x

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import gemmi
import cupy as cp
from pyscf import gto, dft

plt.rcParams["figure.dpi"] = 120

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 103.7 MB/s eta 0:00:00


In [ ]:
# BLOCK 2 — Paths + structure registry

BASE_DIR   = Path("/content")
STRUCT_DIR = BASE_DIR / "ispH_structures"
LAP_DIR    = BASE_DIR / "ispH_stretch_npz"   # for Mode-B (stretch)
SHEAR_DIR  = BASE_DIR / "ispH_shear_npz"     # for Mode-S (cube shear)

for d in (STRUCT_DIR, LAP_DIR, SHEAR_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Map labels to CIF filenames + cluster type (Fe4S4 or Fe3S4)
# Fe3S4 ones we'll add at the end; for now they are marked but you can skip them.
STRUCT_REGISTRY = {
    "3ZGL_TMBPP"    : {"cif": "3ZGL.cif", "cluster": "fe4s4"},
    "3ZGN_AMBPP"    : {"cif": "3ZGN.cif", "cluster": "fe4s4"},
    "3KE8_HMBPP"    : {"cif": "3KE8.cif", "cluster": "fe4s4"},  # CTRL
    "3KE9_ALT"      : {"cif": "3KE9.cif", "cluster": "fe4s4"},
    "3URK_PROP"     : {"cif": "3URK.cif", "cluster": "fe4s4"},
    "3UTC_HBPP"     : {"cif": "3UTC.cif", "cluster": "fe4s4"},
    "3UV6_ALZ"      : {"cif": "3UV6.cif", "cluster": "fe4s4"},
    "3UV7_ALZ"      : {"cif": "3UV7.cif", "cluster": "fe4s4"},  # when ready

    # Fe3S4 / mutants — to be run at the end:
    "3SZU_E126Q"    : {"cif": "3SZU.cif", "cluster": "fe3s4"},
    "3KEF_alt"      : {"cif": "3KEF.cif", "cluster": "fe3s4"},
    "3KEL_alt"      : {"cif": "3KEL.cif", "cluster": "fe3s4"},
    "3KEM_alt"      : {"cif": "3KEM.cif", "cluster": "fe3s4"},
    "3UTD_alt"      : {"cif": "3UTD.cif", "cluster": "fe3s4"},
    "3UV3_alt"      : {"cif": "3UV3.cif", "cluster": "fe3s4"},
}

# Helper to copy CIFs from /content into STRUCT_DIR if you upload them there.
import shutil, os

for label, meta in STRUCT_REGISTRY.items():
    cif_name = meta["cif"]
    src = BASE_DIR / cif_name
    dst = STRUCT_DIR / cif_name
    if src.exists() and not dst.exists():
        print(f"Copying {src} → {dst}")
        shutil.copy(str(src), str(dst))

Copying /content/3ZGL.cif → /content/ispH_structures/3ZGL.cif
Copying /content/3ZGN.cif → /content/ispH_structures/3ZGN.cif
Copying /content/3KE8.cif → /content/ispH_structures/3KE8.cif
Copying /content/3KE9.cif → /content/ispH_structures/3KE9.cif
Copying /content/3URK.cif → /content/ispH_structures/3URK.cif
Copying /content/3UTC.cif → /content/ispH_structures/3UTC.cif
Copying /content/3UV6.cif → /content/ispH_structures/3UV6.cif
Copying /content/3UV7.cif → /content/ispH_structures/3UV7.cif
Copying /content/3SZU.cif → /content/ispH_structures/3SZU.cif
Copying /content/3KEF.cif → /content/ispH_structures/3KEF.cif
Copying /content/3KEL.cif → /content/ispH_structures/3KEL.cif
Copying /content/3KEM.cif → /content/ispH_structures/3KEM.cif
Copying /content/3UTD.cif → /content/ispH_structures/3UTD.cif
Copying /content/3UV3.cif → /content/ispH_structures/3UV3.cif


In [ ]:
# BLOCK 3 — read CIF + extract Fe/S connectivity clusters

def read_structure_any(path: Path):
    print(f"Reading {path}")
    return gemmi.read_structure(str(path))

def get_fe_s_clusters(struct, cutoff=3.0):
    """Return list of connectivity clusters containing Fe/S atoms."""
    atoms = []
    for model in struct:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element.name.upper() in ("FE", "S"):
                        atoms.append(atom)
    if not atoms:
        raise RuntimeError("No Fe or S atoms found")

    coords = np.array([[a.pos.x, a.pos.y, a.pos.z] for a in atoms], float)
    n = len(atoms)
    adj = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if np.linalg.norm(coords[i] - coords[j]) <= cutoff:
                adj[i].append(j)
                adj[j].append(i)

    clusters = []
    visited = [False]*n
    for i in range(n):
        if visited[i]:
            continue
        stack = [i]
        comp = []
        visited[i] = True
        while stack:
            k = stack.pop()
            comp.append(atoms[k])
            for nb in adj[k]:
                if not visited[nb]:
                    visited[nb] = True
                    stack.append(nb)
        clusters.append(comp)
    print("  Found", len(clusters), "Fe/S clusters")
    return clusters

In [ ]:
# BLOCK 4 — Select Fe4S4 or Fe3S4 cluster + convert to PySCF atoms

def select_fe_cluster(clusters, prefer="fe4s4"):
    """
    Prefer Fe4S4; if none found, fallback to Fe3S4.
    prefer can be 'fe4s4' or 'fe3s4' to bias, but we always try both.
    """
    def filter_clusters(clusters, n_fe, min_s):
        out = []
        for comp in clusters:
            nfe = sum(1 for a in comp if a.element.name.upper()=="FE")
            ns  = sum(1 for a in comp if a.element.name.upper()=="S")
            if nfe == n_fe and ns >= min_s:
                out.append(comp)
        return out

    # try preferred cluster type
    if prefer.lower() == "fe4s4":
        cand_main = filter_clusters(clusters, 4, 4)
        cand_alt  = filter_clusters(clusters, 3, 4)
        main_tag, alt_tag = "Fe4S4", "Fe3S4"
    else:
        cand_main = filter_clusters(clusters, 3, 4)
        cand_alt  = filter_clusters(clusters, 4, 4)
        main_tag, alt_tag = "Fe3S4", "Fe4S4"

    target = None
    tag    = None
    if cand_main:
        target = cand_main
        tag    = main_tag
    elif cand_alt:
        target = cand_alt
        tag    = alt_tag
    else:
        raise RuntimeError("No Fe4S4 or Fe3S4 cluster found")

    # pick cluster closest to global Fe/S centroid
    all_atoms  = [a for c in target for a in c]
    all_coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in all_atoms], float)
    global_cent = all_coords.mean(axis=0)

    def centroid(comp):
        coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in comp], float)
        return coords.mean(axis=0)

    best = None
    best_d = 1e9
    for comp in target:
        c = centroid(comp)
        d = np.linalg.norm(c - global_cent)
        if d < best_d:
            best, best_d = comp, d

    print(f"  Selected {tag} cluster with", len(best), "atoms")
    return best, tag

def atoms_to_pyscf(cluster_atoms):
    """Shift cluster to centroid and convert to PySCF atom list."""
    coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in cluster_atoms], float)
    centre = coords.mean(axis=0)
    coords -= centre
    out = []
    for atom,(x,y,z) in zip(cluster_atoms, coords):
        sym = atom.element.name.capitalize()
        out.append((sym, (float(x), float(y), float(z))))
    return out

In [ ]:
# BLOCK 5 — Cube shear distortion (Mode-S, no stretch)

import numpy as np

def apply_cube_shear(pys_atoms, shear=0.20):
    """
    Mode-S: Fe4S4 'cube shear' mode.

    Simple, explicit version:
      - Compute cluster centroid
      - For atoms with x >= 0: shift +shear in x
      - For atoms with x < 0:  shift -shear in x

    This slides one 'half' of the cubane against the other,
    without deliberately stretching individual Fe–S bonds.

    Parameters
    ----------
    pys_atoms : list of (symbol, (x,y,z)) in Å
    shear     : float, Å displacement to apply to each half

    Returns
    -------
    new_atoms : list of (symbol, (x',y',z')) in Å
    """
    symbols = [a[0] for a in pys_atoms]
    coords  = np.array([a[1] for a in pys_atoms], float)

    # 1. Cluster centroid
    centroid = coords.mean(axis=0)

    # 2. Shift into centroid frame
    shifted = coords - centroid

    # 3. Apply shear: one half +shear, other half -shear
    new_shifted = shifted.copy()
    for i, (x, y, z) in enumerate(shifted):
        if x >= 0.0:
            new_shifted[i, 0] += shear
        else:
            new_shifted[i, 0] -= shear

    # 4. Back to global coords
    new_coords = new_shifted + centroid

    new_atoms = []
    for sym, (x, y, z) in zip(symbols, new_coords):
        new_atoms.append((sym, (float(x), float(y), float(z))))
    return new_atoms

In [ ]:
# supposedly the proper shear block 6
def apply_cube_shear(pys_atoms, shear=0.20):
    """
    Apply a true cubic shear to the Fe4S4 cubane:
    - translate one Fe2S2 face by +shear in x
    - translate the opposite Fe2S2 face by -shear in x
    """

    symbols = [a[0] for a in pys_atoms]
    coords  = np.array([a[1] for a in pys_atoms], float)

    # Compute centroid
    centroid = coords.mean(axis=0)

    # Shift into centroid frame
    shifted = coords - centroid

    # Determine sign based on x coordinate
    new_coords = shifted.copy()
    for i, pos in enumerate(shifted):
        if pos[0] >= 0:
            new_coords[i, 0] += shear
        else:
            new_coords[i, 0] -= shear

    # Shift back
    new_coords += centroid

    new_atoms = []
    for sym, pos in zip(symbols, new_coords):
        new_atoms.append((sym, (float(pos[0]), float(pos[1]), float(pos[2]))))

    return new_atoms

In [ ]:
# BLOCK 7 — GPU Laplacian + HOMO orbital + density

L_BOX  = 3.0
N_GRID = 32

def compute_abs_laplacian_gpu(mol, mf, L=L_BOX, N=N_GRID):
    """Compute |∇²ρ| on a cubic grid using CuPy."""
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    print("  Evaluating AO and density on CPU ...")
    ao = dft.numint.eval_ao(mol, coords)
    dm = mf.make_rdm1()
    rho = dft.numint.eval_rho(mol, ao, dm)
    rho_grid = rho.reshape(N, N, N)

    dx = xs[1]-xs[0]
    dy = ys[1]-ys[0]
    dz = zs[1]-zs[0]

    rho_cp = cp.asarray(rho_grid)

    print("  Computing Laplacian on GPU ...")
    d2x = (cp.roll(rho_cp, -1, 0) - 2*rho_cp + cp.roll(rho_cp, 1, 0)) / (dx*dx)
    d2y = (cp.roll(rho_cp, -1, 1) - 2*rho_cp + cp.roll(rho_cp, 1, 1)) / (dy*dy)
    d2z = (cp.roll(rho_cp, -1, 2) - 2*rho_cp + cp.roll(rho_cp, 1, 2)) / (dz*dz)

    lap_cp = d2x + d2y + d2z
    kappa_cp = cp.abs(lap_cp).ravel()
    return cp.asnumpy(kappa_cp).astype(np.float64)

def compute_homo_orbital(mol, mf, L=L_BOX, N=N_GRID):
    """
    Compute HOMO orbital ψ(r) and density |ψ|^2 on the same grid.
    Uses α-spin HOMO of UKS.
    """
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    ao = dft.numint.eval_ao(mol, coords)

    mo_coeff_a = mf.mo_coeff[0]
    mo_occ_a   = mf.mo_occ[0]
    occ_idx = np.where(mo_occ_a > 0.5)[0]
    if len(occ_idx) == 0:
        raise RuntimeError("No occupied alpha orbitals for HOMO")
    homo_idx = int(occ_idx[-1])

    c   = mo_coeff_a[:, homo_idx]
    psi = ao.dot(c)          # ψ(r)
    rho = np.abs(psi)**2     # |ψ|^2

    return psi.astype(np.complex128), rho.astype(np.float64)

In [ ]:
# pre8 PATCH — define SCF, Laplacian, and HOMO for shear notebook

from pyscf import gto, dft
import numpy as np
import cupy as cp

L_BOX  = 3.0
N_GRID = 32

def run_scf_sto3g(mol_atoms, spin=4):
    """
    UKS PBE / STO-3G for Fe-S cluster.
    spin=4 (S=2) is a reasonable high-spin guess for Fe4S4.
    """
    mol = gto.Mole()
    mol.build(
        atom=mol_atoms,
        basis="sto-3g",
        charge=0,
        spin=spin,
        verbose=3,
    )
    mf = dft.UKS(mol)
    mf.xc = "PBE"
    mf.max_cycle = 80
    mf.diis_space = 12
    mf.level_shift = 0.3
    mf.init_guess = "minao"
    print(f"  Running UKS/STO-3G with S={spin/2} ...")
    e = mf.kernel()
    if mf.converged:
        print("  SCF converged, E =", e)
        return mol, mf
    print("  SCF did not converge, trying Newton ...")
    mf2 = mf.newton()
    e2 = mf2.kernel()
    if mf2.converged:
        print("  Newton converged, E =", e2)
        return mol, mf2
    raise RuntimeError("SCF did not converge (UKS + Newton)")

def compute_abs_laplacian_gpu(mol, mf, L=L_BOX, N=N_GRID):
    """
    Compute |∇²ρ| on a cubic grid using CuPy.
    """
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    print("  Evaluating AO and density on CPU ...")
    ao = dft.numint.eval_ao(mol, coords)
    dm = mf.make_rdm1()
    rho = dft.numint.eval_rho(mol, ao, dm)
    rho_grid = rho.reshape(N, N, N)

    dx = xs[1]-xs[0]
    dy = ys[1]-ys[0]
    dz = zs[1]-zs[0]

    rho_cp = cp.asarray(rho_grid)

    print("  Computing Laplacian on GPU ...")
    d2x = (cp.roll(rho_cp, -1, 0) - 2*rho_cp + cp.roll(rho_cp, 1, 0)) / (dx*dx)
    d2y = (cp.roll(rho_cp, -1, 1) - 2*rho_cp + cp.roll(rho_cp, 1, 1)) / (dy*dy)
    d2z = (cp.roll(rho_cp, -1, 2) - 2*rho_cp + cp.roll(rho_cp, 1, 2)) / (dz*dz)

    lap_cp   = d2x + d2y + d2z
    kappa_cp = cp.abs(lap_cp).ravel()
    return cp.asnumpy(kappa_cp).astype(np.float64)

def compute_homo_orbital(mol, mf, L=L_BOX, N=N_GRID):
    """
    Compute HOMO orbital ψ(r) and density |ψ|^2 on the same grid.
    Uses α-spin HOMO of UKS.
    """
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    ao = dft.numint.eval_ao(mol, coords)

    mo_coeff_a = mf.mo_coeff[0]
    mo_occ_a   = mf.mo_occ[0]
    occ_idx = np.where(mo_occ_a > 0.5)[0]
    if len(occ_idx) == 0:
        raise RuntimeError("No occupied alpha orbitals for HOMO")
    homo_idx = int(occ_idx[-1])

    c   = mo_coeff_a[:, homo_idx]
    psi = ao.dot(c)          # ψ(r)
    rho = np.abs(psi)**2     # |ψ|^2

    return psi.astype(np.complex128), rho.astype(np.float64)

In [ ]:
# BLOCK 8 — run SCF + GPU Laplacian + HOMO (cube shear only)

from pathlib import Path
import numpy as np

def run_structure_shear(label, shear=0.20):
    meta = STRUCT_REGISTRY[label]
    cif_name = meta["cif"]
    prefer   = meta["cluster"]
    cif_path = STRUCT_DIR / cif_name
    if not cif_path.exists():
        raise FileNotFoundError(f"{cif_path} not found")

    print(f"\n=== {label} ({cif_name}) [{prefer}] — Mode-S (cube shear) ===")
    st       = read_structure_any(cif_path)
    clusters = get_fe_s_clusters(st, cutoff=3.0)
    core, tag = select_fe_cluster(clusters, prefer=prefer)
    pys_base  = atoms_to_pyscf(core)

    # BASE
    print("  SCF: BASE")
    mol_b, mf_b = run_scf_sto3g(pys_base, spin=4)
    k_base      = compute_abs_laplacian_gpu(mol_b, mf_b)
    psi_b, rho_b = compute_homo_orbital(mol_b, mf_b)

    # SHEAR (Mode-S)
    pys_shear = apply_cube_shear(pys_base, shear=shear)
    print("  SCF: SHEAR geometry")
    mol_s, mf_s = run_scf_sto3g(pys_shear, spin=4)
    k_shear     = compute_abs_laplacian_gpu(mol_s, mf_s)
    psi_s, rho_s = compute_homo_orbital(mol_s, mf_s)

    out_npz = LAP_DIR / f"{label}_shear_sto3g.npz"
    np.savez_compressed(
        out_npz,
        kappa_base   = k_base,
        kappa_mode   = k_shear,     # 'mode' = shear now
        homo_base    = rho_b,
        homo_mode    = rho_s,
        homo_psi_base= psi_b,
        homo_psi_mode= psi_s,
        label        = label,
        cluster_tag  = tag,
        mode         = "shear",
    )
    print("  Saved NPZ →", out_npz)

# EXAMPLE: apply shear to Fe4S4-only set
STRUCTS_SHEAR = [
    "3ZGL_TMBPP",
    "3ZGN_AMBPP",
    "3KE8_HMBPP",
    "3KE9_ALT",
    "3URK_PROP",
    "3UTC_HBPP",
    "3UV6_ALZ",
     "3UV7_ALZ",   # uncomment when you have 3UV7.cif in place
]

for label in STRUCTS_SHEAR:
    run_structure_shear(label, shear=0.20)


=== 3ZGL_TMBPP (3ZGL.cif) [fe4s4] — Mode-S (cube shear) ===
Reading /content/ispH_structures/3ZGL.cif
  Found 16 Fe/S clusters
  Selected Fe4S4 cluster with 11 atoms
  SCF: BASE
  Running UKS/STO-3G with S=2.0 ...
SCF not converged.
SCF energy = -7686.14971307134 after 80 cycles  <S^2> = 6.145012  2S+1 = 5.0576722
  SCF did not converge, trying Newton ...


In [ ]:
# BLOCK 9 — Δteeth(Δr) functions and composites

NBINS_LIST = [
    1,3,5,7,9,10,11,12,13,14,15,16,17,18,19,
    20,21,22,23,24,25,26,27,28,29,
    30,31,32,33,34,35,36,37,38,39,
    40,41,42,43,44,45,46,47,48,49,
    50,51,52,53,54,55,56,57,58,59,
    60,65,70,71,73,75,77,78,79,80,
    81,82,83,84,85,87,88,89,90,
    91,92,93,94,95,96,97,98,99,100,
    102,103,105,106,107,109,
    110,111,112,113,114,115,116,117,118,119,120,
    121,122,123,124,125
]
L_EFF = 3.0
TRAP_THRESHOLD = 1e-9

def count_traps(mask: np.ndarray) -> int:
    n = len(mask); i = 0; k = 0; inside = False
    while i < n:
        if mask[i] and not inside:
            k += 1
            inside = True
        elif not mask[i] and inside:
            inside = False
        i += 1
    return k

def dteeth_from_npz(path: Path):
    d = np.load(path, allow_pickle=True)
    kb = d["kappa_base"].ravel()
    km = d["kappa_mode"].ravel()
    kmin = min(float(kb.min()), float(km.min()))
    kmax = max(float(kb.max()), float(km.max()))
    dr_list = []
    dt_list = []
    for nb in NBINS_LIST:
        edges = np.linspace(kmin, kmax, nb+1)
        hb,_ = np.histogram(kb, bins=edges, density=True)
        hm,_ = np.histogram(km, bins=edges, density=True)
        mb = hb < TRAP_THRESHOLD
        mm = hm < TRAP_THRESHOLD
        n_b = count_traps(mb)
        n_m = count_traps(mm)
        dr = L_EFF / nb
        dr_list.append(dr)
        dt_list.append(n_m - n_b)
    return np.array(dr_list), np.array(dt_list), d["label"], d["mode"]

def composite_plot(dir_path: Path, title_suffix="stretch/shear"):
    curves = {}
    for npz in sorted(dir_path.glob("*_sto3g.npz")):
        dr,dt,lab,mode = dteeth_from_npz(npz)
        curves[f"{lab}_{mode}"] = (dr,dt)

    plt.figure(figsize=(11,6))
    for key,(dr,dt) in curves.items():
        idx = np.argsort(dr)
        drs = dr[idx]
        dts = dt[idx]
        mask = (drs >= 2e-2) & (drs <= 1e-1)
        plt.plot(drs[mask], dts[mask], "-o", ms=3, label=key)

    plt.axhline(0, color="k", lw=1)
    plt.xscale("log")
    plt.xlim(2e-2, 1e-1)
    plt.ylim(-12, 12)
    plt.xlabel("Δr (Å, log scale)")
    plt.ylabel("Δteeth (distorted − base)")
    plt.title(f"GQR-15 Δteeth(Δr) — {title_suffix}")
    plt.grid(True, which="both", alpha=0.25)
    plt.legend(fontsize=8)
    plt.tight_layout()
    plt.show()



composite_plot(LAP_DIR, title_suffix="IspH Fe–S, Mode-S (cube shear)")

In [ ]:
# BLOCK 10 — HOMO visualisation helpers

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

def plot_homo_density_cloud(npz_path: Path, title_prefix="", frac=0.03, N=N_GRID):
    d = np.load(npz_path, allow_pickle=True)
    rho_b = d["homo_base"].reshape(N,N,N)
    rho_m = d["homo_mode"].reshape(N,N,N)

    xs = np.linspace(-L_BOX, L_BOX, N)
    ys = np.linspace(-L_BOX, L_BOX, N)
    zs = np.linspace(-L_BOX, L_BOX, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")

    Xf = X.ravel(); Yf = Y.ravel(); Zf = Z.ravel()
    hb = rho_b.ravel(); hm = rho_m.ravel()

    thr_b = np.quantile(hb, 1.0 - frac)
    thr_m = np.quantile(hm, 1.0 - frac)
    mask_b = hb >= thr_b
    mask_m = hm >= thr_m

    fig = plt.figure(figsize=(10,4))

    ax1 = fig.add_subplot(1,2,1, projection="3d")
    p1 = ax1.scatter(Xf[mask_b], Yf[mask_b], Zf[mask_b], c=hb[mask_b],
                     s=8, alpha=0.6, cmap="viridis")
    ax1.set_title(f"{title_prefix} HOMO |ψ|^2 — BASE")
    fig.colorbar(p1, ax=ax1, shrink=0.7, label="|ψ|^2")

    ax2 = fig.add_subplot(1,2,2, projection="3d")
    p2 = ax2.scatter(Xf[mask_m], Yf[mask_m], Zf[mask_m], c=hm[mask_m],
                     s=8, alpha=0.6, cmap="viridis")
    ax2.set_title(f"{title_prefix} HOMO |ψ|^2 — Mode-B/S")
    fig.colorbar(p2, ax=ax2, shrink=0.7, label="|ψ|^2")

    for ax in (ax1,ax2):
        ax.set_xlabel("x (Å)")
        ax.set_ylabel("y (Å)")
        ax.set_zlabel("z (Å)")

    plt.tight_layout()
    plt.show()

def plot_homo_blue_red_slice(npz_path: Path, title_prefix="", N=N_GRID):
    d = np.load(npz_path, allow_pickle=True)
    psi_b = d["homo_psi_base"]
    psi_m = d["homo_psi_mode"]
    pb = psi_b.reshape(N,N,N)
    pm = psi_m.reshape(N,N,N)
    z_idx = N//2
    slice_b = pb[:,:,z_idx].real
    slice_m = pm[:,:,z_idx].real
    vmax = max(abs(slice_b).max(), abs(slice_m).max())
    vlim = vmax if vmax>0 else 1.0

    fig, axes = plt.subplots(1,2,figsize=(8,4))
    im0 = axes[0].imshow(slice_b, origin='lower', cmap='bwr', vmin=-vlim, vmax=vlim)
    axes[0].set_title(f"{title_prefix} ψ — BASE")
    axes[0].axis('off')
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(slice_m, origin='lower', cmap='bwr', vmin=-vlim, vmax=vlim)
    axes[1].set_title(f"{title_prefix} ψ — distorted")
    axes[1].axis('off')
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

# Example usage:
# plot_homo_density_cloud(LAP_DIR/"3ZGN_AMBPP_stretch_sto3g.npz", "3ZGN_AMBPP")
# plot_homo_blue_red_slice(LAP_DIR/"3ZGN_AMBPP_stretch_sto3g.npz", "3ZGN_AMBPP")

In [ ]:
# BLOCK 8 — driver to run stretch / shear for a structure and save NPZs

def run_structure(label, mode="stretch", gamma_shear=0.10):
    meta = STRUCT_REGISTRY[label]
    cif_name  = meta["cif"]
    prefer    = meta["cluster"]
    cif_path  = STRUCT_DIR / cif_name
    if not cif_path.exists():
        raise FileNotFoundError(f"{cif_path} not found")

    print(f"\n=== {label} ({cif_name}) [{prefer}] / Mode-{mode.upper()[0]} ===")
    st       = read_structure_any(cif_path)
    clusters = get_fe_s_clusters(st, cutoff=3.0)
    core, tag = select_fe_cluster(clusters, prefer=prefer)
    pys_base = atoms_to_pyscf(core)

    # BASE
    print("  SCF: BASE")
    mol_b, mf_b = run_scf_sto3g(pys_base, spin=4)
    k_base      = compute_abs_laplacian_gpu(mol_b, mf_b)
    psi_b, rho_b = compute_homo_orbital(mol_b, mf_b)

    # DISTORTED
    if mode.lower() == "stretch":
        pys_dist = apply_mode_b(pys_base, shift=MODE_B_SHIFT)
        out_dir  = LAP_DIR
        suffix   = "stretch"
    elif mode.lower() == "shear":
        pys_dist = apply_cube_shear(pys_base, gamma=gamma_shear)
        out_dir  = SHEAR_DIR
        suffix   = "shear"
    else:
        raise ValueError("mode must be 'stretch' or 'shear'")

    print(f"  SCF: {mode.upper()} geometry")
    mol_d, mf_d = run_scf_sto3g(pys_dist, spin=4)
    k_dist      = compute_abs_laplacian_gpu(mol_d, mf_d)
    psi_d, rho_d = compute_homo_orbital(mol_d, mf_d)

    out_npz = out_dir / f"{label}_{suffix}_sto3g.npz"
    np.savez_compressed(
        out_npz,
        kappa_base=k_base,
        kappa_mode=k_dist,
        homo_base=rho_b,
        homo_mode=rho_d,
        homo_psi_base=psi_b,
        homo_psi_mode=psi_d,
        label=label,
        cluster_tag=tag,
        mode=mode,
    )
    print("  Saved NPZ →", out_npz)


# EXAMPLE: run stretch on Fe4S4 set first
STRUCTS_TO_RUN_STRETCH = [
    "3ZGL_TMBPP",
    "3ZGN_AMBPP",
    "3KE8_HMBPP",
    "3KE9_ALT",
    "3URK_PROP",
    "3UTC_HBPP",
    "3UV6_ALZ",
    "3UV7_ALZ",
]

for label in STRUCTS_TO_RUN_STRETCH:
    run_structure(label, mode="stretch")

# Later, for shear:
# for label in STRUCTS_TO_RUN_STRETCH:
#     run_structure(label, mode="shear", gamma_shear=0.10)